In [1]:
import cv2
import numpy as np
import time
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
import pandas as pd
from scipy.ndimage import label
import sys

In [2]:
def contrast_strech(image):
    a = 0
    b = 255
    c = image.min()
    d_max = image.max()
    image_new = image
    image_new[:,:,:] = (image[:,:,:] - c)* ((b-a)/(d_max - c)) + a #(I-c)(b-a/max - min) + a
    return image_new

In [3]:
def convolve_max_filtered(image,N):
    h,w = image.shape
    output = np.zeros((h,w),dtype = np.uint8)
    #filter_new = np.array([[1]*N]*N  ,dtype ="int")  ## N*N filter with 1 in all places
    #need to pad the image according to filter size
    p = int((N-1)/2) #paading of zeros required
    padded_picture = cv2.copyMakeBorder(image.copy(),p,p,p,p,cv2.BORDER_CONSTANT,value = 0)
    for y in np.arange(p,h+p):
        for x in np.arange(p,w+p):
                roi = padded_picture[y-p:y+p+1, x-p:x+p+1]
                max_pixel = np.amax((roi)) #directly finding the max value from surrounding pixels and replacing it
                output[y-p,x-p]= max_pixel
    output = (output*255).astype("uint8")
    return output

In [4]:
def convolve_min_filtered(image,N):
    h,w = image.shape
    output = np.full((h,w),255,dtype = np.uint8)
    #filter_new = np.array([[1]*N]*N  ,dtype ="int")  ## N*N filter with 1 in all places
    #need to pad the image according to filter size
    p = int((N-1)/2) #paading of zeros required
    padded_picture = cv2.copyMakeBorder(image.copy(),p,p,p,p,cv2.BORDER_CONSTANT,value = 255)
    
    for y in np.arange(p,h+p):
        for x in np.arange(p,w+p):
                roi = padded_picture[y-p:y+p+1, x-p:x+p+1]
                min_pixel = np.amin((roi)) #directly finding the max value from surrounding pixels and replacing it
                output[y-p,x-p]= min_pixel
           
    output = (output*255).astype("uint8")
    return output

In [5]:
def remove_background(image,back_img):
    h,w = image.shape
    output = np.zeros((h,w),dtype = "uint8")
    output = np.subtract(image,back_img)
    return output

In [6]:
def run_algo(image,filter_size, M):
    if M == 0:
        max_filter =  convolve_max_filtered(image,filter_size)
        #min_filter =  convolve_min_filtered(max_filter,filter_size)
        final_image = remove_background(image, max_filter)
    else:
        min_filter = convolve_min_filtered(image,filter_size)
        #max_filter = convolve_max_filtered(min_filter,filter_size)
        final_image = remove_background(image, min_filter)
    return final_image

In [7]:
def apply_watershed(img):
    img_array = img.copy()
    distance = ndi.distance_transform_edt(img_array)
    markers = ndi.label(peak_local_max(distance, indices = False, labels = img_array))[0]
    ws_labels = watershed(-distance, markers, mask = img_array)
    print("[INFO] {} unique segments found".format(len(np.unique(ws_labels)) - 1))
    return ws_labels,markers

In [8]:
def image_sharpening(image):
    sigma = 10
    a = 10
    blur_image = cv2.GaussianBlur(image,(0,0),sigma)  #L = I*G
    #H = image - blur_image #H = I - L
    H = cv2.subtract(image,blur_image)
    final_image = H*a + image #o = I+H*a
    return final_image

In [9]:
def gradx(img):
    img = img.astype('int')
    rows, cols = img.shape
    # Use hstack to add back in the columns that were dropped as zeros
    return np.hstack( (np.zeros((rows, 1)), (img[:, 2:] - img[:, :-2])/2.0, np.zeros((rows, 1))) )

def grady(img):
    img = img.astype('int')
    rows, cols = img.shape
    # Use vstack to add back the rows that were dropped as zeros
    return np.vstack( (np.zeros((1, cols)), (img[2:, :] - img[:-2, :])/2.0, np.zeros((1, cols))) )

#Performs fast radial symmetry transform
#img: input image, grayscale
#radii: integer value for radius size in pixels (n in the original paper); also used to size gaussian kernel
#alpha: Strictness of symmetry transform (higher=more strict; 2 is good place to start)
#beta: gradient threshold parameter, float in [0,1]
#stdFactor: Standard deviation factor for gaussian kernel
#mode: BRIGHT, DARK, or BOTH
def frst(img, radii, alpha, beta, stdFactor, mode='BOTH'):
    mode = mode.upper()
    assert mode in ['BRIGHT', 'DARK', 'BOTH']
    dark = (mode == 'DARK' or mode == 'BOTH')
    bright = (mode == 'BRIGHT' or mode == 'BOTH')

    workingDims = tuple((e + 2*radii) for e in img.shape)

    #Set up output and M and O working matrices
    output = np.zeros(img.shape, np.uint8)
    O_n = np.zeros(workingDims, np.int16)
    M_n = np.zeros(workingDims, np.int16)

    #Calculate gradients
    gx = gradx(img)
    gy = grady(img)

    #Find gradient vector magnitude
    gnorms = np.sqrt( np.add( np.multiply(gx, gx) , np.multiply(gy, gy) ) )

    #Use beta to set threshold - speeds up transform significantly
    gthresh = np.amax(gnorms)*beta

    #Find x/y distance to affected pixels
    gpx = np.multiply(np.divide(gx, gnorms, out=np.zeros(gx.shape), where=gnorms!=0), radii).round().astype(int);
    gpy = np.multiply(np.divide(gy, gnorms, out=np.zeros(gy.shape), where=gnorms!=0), radii).round().astype(int);

    #Iterate over all pixels (w/ gradient above threshold)
    for coords, gnorm in np.ndenumerate(gnorms):
        if gnorm > gthresh:
            i, j = coords
            #Positively affected pixel
            if bright:
                ppve = (i+gpx[i,j], j+gpy[i,j])
                O_n[ppve] += 1
                M_n[ppve] += gnorm
            #Negatively affected pixel
            if dark:
                pnve = (i-gpx[i,j], j-gpy[i,j])
                O_n[pnve] -= 1
                M_n[pnve] -= gnorm

    #Abs and normalize O matrix
    O_n = np.abs(O_n)
    O_n = O_n / float(np.amax(O_n))

    #Normalize M matrix
    M_max = float(np.amax(np.abs(M_n)))
    M_n = M_n / M_max

    #Elementwise multiplication
    F_n = np.multiply(np.power(O_n, alpha), M_n)

    #Gaussian blur
    kSize = int( np.ceil( radii / 2 ) )
    kSize = kSize + 1 if kSize % 2 == 0 else kSize

    S = cv2.GaussianBlur(F_n, (kSize, kSize), int( radii * stdFactor ))

    return S

In [10]:
def segmentation_try_2(frame,gray):
        ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        # noise removal
        kernel = np.ones((3,3),np.uint8)
        opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
        # sure background area
        sure_bg = cv2.dilate(opening,kernel,iterations=3)
        # Finding sure foreground area
        dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
        ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
        # Finding unknown region
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg,sure_fg)
        # Marker labelling
        ret, markers = cv2.connectedComponents(sure_fg)
        # Add one to all labels so that sure background is not 0, but 1
        markers = markers+1
        # Now, mark the region of unknown with zero
        markers[unknown==255] = 0
        markers = cv2.watershed(frame,markers)
        #frame_boxes = make_boxes(frame,markers)
        frame[markers == -1] = [255,0,0]
        df = pd.DataFrame(markers)
        df.to_csv("markers.csv")
        return frame

In [11]:
def make_boxes(color_img,image,labels):       
    conts=[]    
    contours=[]
    for label in np.unique(labels):
        # if the label is zero, we are examining the 'background'
        # so simply ignore it
        if label == 0:
            continue
        # otherwise, allocate memory for the label region and draw
        # it on the mask
        mask = np.zeros(gray.shape, dtype="uint8")
        mask[labels == label] = 255
        # detect contours in the mask and grab the largest one
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[-2]
        c = max(cnts, key=cv2.contourArea)
        
#         rect = cv2.minAreaRect(c)
#         box = cv2.boxPoints(rect)
#         box = np.int0(box)  
#         if cv2.contourArea(c) > 50: 
#             #cv2.drawContours(image,c,-1,(0,255,0))
#             cv2.drawContours(image,[box],-1,(128,128,128),thickness = 1)
        hull = cv2.convexHull(c)
        #cv2.drawContours(image,[hull],-1,(0,0,255),thickness = 1)
        contours.append(hull)
    for i in range(len(contours)):
        c = contours[i]

        area = cv2.contourArea(c)

        # Iterate all contours from i+1 to end of list
        for j in range(i+1, len(contours)):
            c2 = contours[j]

            area2 = cv2.contourArea(c2)

            area_sum = area + area2

            # Merge contours together
            tmp = np.vstack((c, c2))
            merged_c = cv2.convexHull(tmp)

            merged_area = cv2.contourArea(merged_c)

            # Replace contours c and c2 by the convex hull of merged c and c2, if total area is increased by no more then 10%
            if merged_area < area_sum*1.1:
                # Replace contour with merged one.
                contours[i] = merged_c
                contours[j] = merged_c
                c = merged_c
                area = merged_area
    ################################################################################


    # Draw new contours in red color
    for c in contours:
        #Ignore small contours
        if cv2.contourArea(c) > 20:
            cv2.drawContours(color_img, [c], -1, (255,0,0), 2, 1)
            cv2.drawContours(image, [c], -1, (128,128,128), 2, 1)
    return image,color_img

In [12]:
def segmentation_try_1(frame,gray):
        image = frame.copy()
        #applying min_max filtering
        #frame = run_algo(gray, filter_size,M)
        kernel_erosion = np.ones((3,3),np.uint8)
        erosion = cv2.erode(gray,kernel_erosion,iterations = 1)
        kernel_dilation = np.ones((5,5), np.uint8)
        img_dilation = cv2.dilate(erosion, kernel_dilation, iterations=1)
        th = cv2.adaptiveThreshold(img_dilation,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        frame = remove_background(th,img_dilation)
        blur = cv2.GaussianBlur(frame,(5,5),0)
        ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        frame = cv2.bitwise_not(th)
        #frame = frst(frame,2,2,0.25,0.25)
        label,markers = apply_watershed(frame)
       # print("labels",label)
       # print("markers",markers)
       # pd.DataFrame(markers).to_csv("markers.csv")
       # pd.DataFrame(label).to_csv("labels.csv")
        frame_boxes,color_img = make_boxes(image,frame,label)
        return color_img,frame_boxes

In [13]:
def segment_on_dt(a, img):
    border = cv2.dilate(img, None, iterations=5)
    border = border - cv2.erode(border, None)

    dt = cv2.distanceTransform(img, 2, 3)
    dt = ((dt - dt.min()) / (dt.max() - dt.min()) * 255).astype(np.uint8)
    _, dt = cv2.threshold(dt, 180, 255, cv2.THRESH_BINARY)
    lbl, ncc = label(dt)
    lbl = lbl * (255 / (ncc + 1))
    # Completing the markers now. 
    lbl[border == 255] = 255

    lbl = lbl.astype(np.int32)
    cv2.watershed(a, lbl)

    lbl[lbl == -1] = 0
    lbl = lbl.astype(np.uint8)
    return 255 - lbl

In [14]:
def preprocessing_different(img,img_gray):
    _, img_bin = cv2.threshold(img_gray, 0, 255,
            cv2.THRESH_OTSU)
    img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN,
            np.ones((3, 3), dtype=int))
    cv2.imshow("morph",img_bin)
    result = segment_on_dt(img, img_bin)
    result[result != 255] = 0
    result = cv2.dilate(result, None)
    img[result == 255] = (0, 0, 255)
    return img

In [16]:
cap = cv2.VideoCapture("./COMP9517_Images/PhC-C2DL-PSC/Sequence 4/t%03d.tif")
filter_size = 3
M = 0
while(1):
    ret, frame = cap.read()
    if (ret) :  
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('original image',gray)
        
        #pre_img = preprocessing_different(frame,gray)
        #cv2.imshow("pre_img",pre_img)
        segmentation_try_1_img,Black = segmentation_try_1(frame,gray)
        
        
        #Colored Image
        cv2.imshow('segmentation_try_1',segmentation_try_1_img)
        #Black Image
        cv2.imshow('black and white',Black)
        
        
        #segmentation_try_2_img = segmentation_try_2(frame,gray)
        #cv2.imshow('segmentation_try_2',segmentation_try_2_img)
        
        #ct.update()
        
        time.sleep(0.5)
        
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    # 'q' to quit playback
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break
        
cap.release()
cv2.destroyAllWindows()

[INFO] 189 unique segments found
[INFO] 179 unique segments found
[INFO] 188 unique segments found
[INFO] 190 unique segments found
[INFO] 191 unique segments found
[INFO] 191 unique segments found
[INFO] 187 unique segments found
[INFO] 194 unique segments found
[INFO] 192 unique segments found
[INFO] 209 unique segments found
[INFO] 186 unique segments found
[INFO] 189 unique segments found
[INFO] 194 unique segments found
[INFO] 192 unique segments found
[INFO] 190 unique segments found
[INFO] 196 unique segments found
[INFO] 189 unique segments found
[INFO] 182 unique segments found
[INFO] 188 unique segments found
[INFO] 186 unique segments found
[INFO] 192 unique segments found
[INFO] 180 unique segments found
[INFO] 203 unique segments found
[INFO] 211 unique segments found
[INFO] 216 unique segments found
[INFO] 204 unique segments found
[INFO] 192 unique segments found
[INFO] 194 unique segments found
[INFO] 219 unique segments found
[INFO] 208 unique segments found
[INFO] 196

[INFO] 881 unique segments found
[INFO] 905 unique segments found
[INFO] 910 unique segments found
[INFO] 910 unique segments found
[INFO] 914 unique segments found
[INFO] 942 unique segments found
[INFO] 946 unique segments found
[INFO] 956 unique segments found
[INFO] 923 unique segments found
[INFO] 937 unique segments found
[INFO] 948 unique segments found
[INFO] 982 unique segments found
[INFO] 965 unique segments found
[INFO] 997 unique segments found
[INFO] 1012 unique segments found
[INFO] 1007 unique segments found
[INFO] 1013 unique segments found
[INFO] 1013 unique segments found
[INFO] 1043 unique segments found
[INFO] 1011 unique segments found
[INFO] 1042 unique segments found
[INFO] 1052 unique segments found
[INFO] 1037 unique segments found
[INFO] 1033 unique segments found
[INFO] 1019 unique segments found
[INFO] 1043 unique segments found
[INFO] 1044 unique segments found
[INFO] 1035 unique segments found
[INFO] 1047 unique segments found
[INFO] 1059 unique segments 

KeyboardInterrupt: 

# DONT RUN FROM HERE.

In [ ]:
ct = CentroidTracker()

In [ ]:
# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

class CentroidTracker():
    def __init__(self, maxDisappeared=50):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()

        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        # check to see if the list of input bounding box rectangles
        # is empty
        if len(rects) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # return early as there are no centroids or tracking info
            # to update
            return self.objects

        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")

        # loop over the bounding box rectangles
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])

        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()

            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]

            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        # return the set of trackable objects
        return self.objects